
## Downloading the data 

### AEM data

The "point of truth" source of the AEM data is under: https://ecat.ga.gov.au/geonetwork/srv/eng/catalog.search#/metadata/82307

```bash
mkdir -p $HOME/data/Lithology/AEM
cd $HOME/data/Lithology/AEM
wget https://d28rz98at9flks.cloudfront.net/82307/SwanCoastalPlain.zip
unzip SwanCoastalPlain.zip
# avoid spaces in folder names:
mv Swan\ Coastal\ Plain Swan_coastal_plains
```

### Elevation

ELVIS - TODO


### Borehole logs

NGIS - TODO


In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import geopandas as gpd


In [ ]:
# Only set to True for co-dev of ela from this use case:
ela_from_source = False
ela_from_source = True

In [ ]:
if ela_from_source:
    if ('ELA_SRC' in os.environ):
        root_src_dir = os.environ['ELA_SRC']
    elif sys.platform == 'win32':
        root_src_dir = r'C:\src\github_jm\pyela'
    else:
        username = os.environ['USER']
        root_src_dir = os.path.join('/home', username, 'src/github_jm/pyela')
    pkg_src_dir = root_src_dir
    sys.path.append(pkg_src_dir)

from ela.textproc import *
from ela.utils import *
from ela.classification import *
from ela.visual import *
from ela.spatial import SliceOperation

## Importing data

There are two main sets of information we need: the borehole lithology logs, and the spatial information in the surface elevation (DEM) and geolocation of a subset of bores around Bungendore. 

In [ ]:
data_path = None

You probably want to explicitly set `data_path` to the location where you put the 'Bungendore' folder e.g:

In [ ]:
#data_path = '/home/myusername/data' # On Linux, if you now have the folder /home/myusername/data/Bungendore
#data_path = r'C:\data\Lithology'  # windows, if you have C:\data\Lithology\Bungendore

Otherwise a fallback for the pyela developer(s)

In [ ]:
if data_path is None:
    if ('ELA_DATA' in os.environ):
        data_path = os.environ['ELA_DATA']
    elif sys.platform == 'win32':
        data_path = r'C:\data\Lithology'
    else:
        username = os.environ['USER']
        data_path = os.path.join('/home', username, 'data')

In [ ]:
aem_datadir = os.path.join(data_path, 'AEM')
scp_datadir = os.path.join(aem_datadir, 'Swan_coastal_plains')
scp_grids_datadir = os.path.join(scp_datadir, 'grids')

## DEM

Let's have a look at the DEM provided as part of the AEM package

In [ ]:
dem = rasterio.open(os.path.join(scp_grids_datadir,'dem/Swan_Coastal_Plain_DEM_Final.ers'))

In [ ]:
dem.crs

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
show(dem,title='Swan Coastal Plain', cmap='terrain',  ax=ax)

This is quite a gap in the data. Odd this should be this way. We will fortunately rely on the data from ELVIS which would not have these odd holes.

Now on to looking at the conductivity slices. 

## Conductivity slices


In [ ]:
cnd_slice_dir = os.path.join(scp_grids_datadir,'cnd')

cnd_000_005 = rasterio.open(os.path.join(cnd_slice_dir,'Swan_Coastal_Plain_CND_000m_to_005m_Final.ers'))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
show(cnd_000_005,title='Conductivity 0-5 metres depth (units?)', cmap='viridis',  ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
show(cnd_000_005,title='Conductivity 0-5 metres depth (units?)', cmap='magma',  ax=ax)